# Amazon UK Search Results

All the necessary imports to run this notebook. Ensure selenium has been installed in an external notebook before running this one.

In [1]:
#imports
import requests
import json
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
#selenium imports
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selectorlib import Extractor
import requests
import json
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#pip install selectorlib

## Loading pre-documented gender stereotyped items

Load in predoc_stereotyped_items.csv. Should return 72 rows.

In [3]:
stereo_toys = pd.read_csv('~/Desktop/485/Data/predoc_stereotyped_items.csv', delimiter =',')
stereo_toys

,BOY,GIRL,NEUTRAL
0,vehicle toys,doll,toy animals
1,sport,domestic toys,books
2,military toys,educational art,educational teaching
3,race cars,clothes,musical games
4,outer space toys,dollhouses,games
...,...,...,...
67,toy rocket,barbie furniture set,NaN
68,soccer ball,NaN,NaN
69,blue ipad,pink ipad,NaN
70,toy robots,NaN,NaN


### read in all_items.txt

In [4]:
with open('all_items.txt') as f:
    all_items = f.read().splitlines()

In [5]:
len(all_items)

166

In [6]:
print(all_items)

['vehicle toys', 'sport', 'military toys', 'race cars', 'outer space toys', 'depots', 'machines', 'doll-humanoid', 'action figures', 'gi joe action figure', 'spiderman', 'guitar', 'castle tent', 'microscope', 'monster trucks', 'bug collection set', 'barn', 'grill', 'dinosaur toy', 'volcano creator', 'pokemon cards', 'space station toys', 'basketball hoop', 'gears', 'train set', 'lincoln logs', 'police station toy', 'airport toy', 'police officer gear', 'dragonballz', 'superhero costumes', 'helicopter', 'racetrack', 'remote control car', 'football', 'hockey stick', 'sword toy', 'miniature guns', 'truck set', 'remote control helicopter', 'sully costume', 'woody valentine', 'spiderman costume', 'batman costume', 'ninja costume', 'wwe action figure', 'action figure', 'toy gun', 'construction games', 'war toys', 'toy vehicles', 'educational material', 'nerf guns', 'construction toys', 'car toys', 'video games', 'building blocks', 'dinosaur toys', 'lego cars', 'lego toys', 'computer games', 

### create a trial list of all_items

In [7]:
trial = all_items[160:]
trial

['legos', 'scooter', 'drum set', 'puzzles', 'board games', 'rock painting']

In [8]:
generic = ['toys', 'books', 'learning material', 'games', 'sports']

In [9]:
gender = ['boys', 'girls', 'neutral']

## Scraping Data From Search Results

### Getting asin of the retrieved products

ensure xpath is correct

In [10]:
def asin(driver):
    asin_list = []
    for index in range(1,10):
        asins = driver.find_elements("xpath","//div[@data-asin]")
        for asin in asins:
            #print(asin)
            asin_list.append(asin.get_attribute('data-asin'))
    return asin_list

### get information (title) of retirved products

In [11]:
from selenium.webdriver.common.by import By
def item_info(driver):
    item = []
    elem = driver.find_elements(By.CSS_SELECTOR,'h2')
    for i in elem:
        item.append(i.text)
    return item

### Getting product link of the retrievd products
We did not use this codeblock since we had the asin to use to compare products. Depending on the e-commerce site link might be necessary.

In [12]:
def item_link(driver):
    href = []
    links = driver.find_elements('xpath',"//h2//a[@href]")
    for link in links:
        href.append(link.get_attribute('href'))
    return href

### Running queries for both boy and girls and neutral (kids)

In [13]:
def search(item, who):
    if who == 'neutral':
        query = item + ' for '+'kids'
    else:
        query = item + ' for '+who
    driver.get(f'https://www.amazon.co.uk/s?k={query}&ref=nb_sb_noss')
    list_asin = asin(driver)
    item_list = item_info(driver)
    item_page = item_link(driver)
    return (list_asin, item_list), item_page

### create DataFrame to load scraped data into
Since we did not use item_link above we do not need the columns2 dataframe.

In [16]:
columns1 = ['gender', 'query', 'result']
qr = pd.DataFrame(columns = columns1)
columns2 = ['gender', 'query', 'href']
qr_link = pd.DataFrame(columns = columns2)
#qr_link

## running queries in a loop

Below is code to run with a trial set of 6 items to ensure your code runs before running to code with all_itmes with contains 160 items.

In [17]:
#trial run
import time
import warnings
warnings.filterwarnings('ignore')
driver = webdriver.Chrome(ChromeDriverManager().install())
data1 = []
data2 = []
item = ''
for item in trial:
    for g in gender:
        result, link = search(item, g)
        values1 = [g, item, result]
        values2 = [g, item, link]
        zipped1 = zip(columns1, values1)
        zipped2 = zip(columns2, values2)
        a_dictionary1 = dict(zipped1)
        a_dictionary2 = dict(zipped2)
        #print(a_dictionary)
        time.sleep(1.5)
        data1.append(a_dictionary1)
        data2.append(a_dictionary2)
driver.close()

Run this code block with all items once the trial code blocks runs successfully. This will take some time.

In [33]:
#all_items run
import time
import warnings
warnings.filterwarnings('ignore')
driver = webdriver.Chrome(ChromeDriverManager().install())
data1 = []
data2 = []
item = ''
for item in all_items:
    for g in gender:
        result, link = search(item, g)
        values1 = [g, item, result]
        values2 = [g, item, link]
        zipped1 = zip(columns1, values1)
        zipped2 = zip(columns2, values2)
        a_dictionary1 = dict(zipped1)
        a_dictionary2 = dict(zipped2)
        #print(a_dictionary)
        time.sleep(1.5)
        data1.append(a_dictionary1)
        data2.append(a_dictionary2)
driver.close()

KeyboardInterrupt: 

In [18]:
qr = qr.append(data1, True)
qr

,gender,query,result
0,boys,legos,"([, B09FM7NFY7, B088QCH3DQ, B07FNWMBML, B000CI..."
1,girls,legos,"([, B09FM7NFY7, B09FM6DRLB, B097BFPNCJ, B07KS7..."
2,neutral,legos,"([, B07FNWMBML, B09FM6DRLB, B0BPMX9PPK, B09FM7..."
3,boys,scooter,"([, B08LVPGB2N, B08Q3D5YF2, B08GS7NYLF, B08BFD..."
4,girls,scooter,"([, B07RK2FTTS, B08LVPGB2N, B09MFFXC6W, B08BLL..."
5,neutral,scooter,"([, B08Z6WX2XC, B09XB9F4QF, B09MFFXC6W, B0BDLC..."
6,boys,drum set,"([, B0BJ26TBQ7, B099DL3L8P, B0B5MKSZ1R, B09MDG..."
7,girls,drum set,"([, B0BJ26TBQ7, B0B5MKSZ1R, B099DL3L8P, B09MRL..."
8,neutral,drum set,"([, B099DL3L8P, B0B4V3T25K, B0B48S9KCP, B0B5MK..."
9,boys,puzzles,"([, B076WVK44C, B096YDZYSF, B079SXW3WD, B0B466..."


In [19]:
qr_link = qr_link.append(data2, True)
qr_link

,gender,query,href
0,boys,legos,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
1,girls,legos,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
2,neutral,legos,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
3,boys,scooter,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
4,girls,scooter,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
5,neutral,scooter,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
6,boys,drum set,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
7,girls,drum set,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
8,neutral,drum set,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...
9,boys,puzzles,[https://www.amazon.co.uk/sspa/click?ie=UTF8&s...


In [20]:
first20 = pd.DataFrame()

In [21]:
first20 = first20.append(qr, ignore_index =True)

In [22]:
first20

,gender,query,result
0,boys,legos,"([, B09FM7NFY7, B088QCH3DQ, B07FNWMBML, B000CI..."
1,girls,legos,"([, B09FM7NFY7, B09FM6DRLB, B097BFPNCJ, B07KS7..."
2,neutral,legos,"([, B07FNWMBML, B09FM6DRLB, B0BPMX9PPK, B09FM7..."
3,boys,scooter,"([, B08LVPGB2N, B08Q3D5YF2, B08GS7NYLF, B08BFD..."
4,girls,scooter,"([, B07RK2FTTS, B08LVPGB2N, B09MFFXC6W, B08BLL..."
5,neutral,scooter,"([, B08Z6WX2XC, B09XB9F4QF, B09MFFXC6W, B0BDLC..."
6,boys,drum set,"([, B0BJ26TBQ7, B099DL3L8P, B0B5MKSZ1R, B09MDG..."
7,girls,drum set,"([, B0BJ26TBQ7, B0B5MKSZ1R, B099DL3L8P, B09MRL..."
8,neutral,drum set,"([, B099DL3L8P, B0B4V3T25K, B0B48S9KCP, B0B5MK..."
9,boys,puzzles,"([, B076WVK44C, B096YDZYSF, B079SXW3WD, B0B466..."


In [23]:
len(first20.loc[0]['result'][1])

66

In [24]:
first20 = pd.read_csv('item_results.csv')
first20

,gender,query,result
0,boys,NaN,"(['', 'B09FT5P8RK', 'B079SXW3WD', 'B08T6NCLW7'..."
1,girls,NaN,"(['', 'B0BC2DHXCY', 'B07YFGX53N', 'B07MQ1GVFB'..."
2,neutral,NaN,"(['', '', 'B08F71TW93', 'B07752FRTL', 'B079SXW..."
3,boys,legos,"(['', 'B097BFPNCJ', 'B07W6Q952Z', 'B07FP6ZWN3'..."
4,girls,legos,"(['', 'B09QFQFY8D', 'B0BLCVCJ5M', 'B09LLT66QQ'..."
...,...,...,...
58,girls,board games,"(['', 'B09S3ZX32K', 'B086XJTLRY', 'B0BKH84MR1'..."
59,neutral,board games,"(['', 'B09S3ZX32K', 'B09X1P468K', 'B078S8D27R'..."
60,boys,rock painting,"(['', 'B08XQ1R4LT', 'B08XK67TRQ', 'B08RDCZWC7'..."
61,girls,rock painting,"(['', 'B08XQ1R4LT', 'B0B2HSPDRC', 'B0B4NWCJ3J'..."


In [25]:
first20.to_csv('az_uk_search_results.csv', index = False)

In [26]:
type(first20.loc[0]['result'])

str